In [1]:
import urllib3
import re
from bs4 import BeautifulSoup
import os
import justext

In [2]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

http = urllib3.PoolManager(10, headers=user_agent)


class Crawler:
    
    def __init__(self, corpus_path, max_files, seed_url, url_pattern):
        self.corpus_path = corpus_path
        self.max_files = max_files
        self.seed_url = seed_url
        self.url_pattern = url_pattern
        self.visited_links = {}
        self.to_be_visited = []
        
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()
        
        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        with open("{}/{}.txt".format(self.corpus_path, url.replace(".", "_").replace("/","-")), "w") as output_file:
            for paragraph in paragraphs:
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')
        links = [link.get('href') for link in soup.findAll('a', attrs={'href': re.compile(self.url_pattern)})]

        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links[next_link] = None
        return next_link

In [3]:
crawler_tecnologia = Crawler("data/corpora/tecnologia", 50, "https://tecnoblog.net/281950/", "^https://tecnoblog\.net/\d+")
crawler_politica = Crawler("data/corpora/politica", 50, "http://blogs.opovo.com.br/politica/",
                           "^http://blogs\.opovo\.com\.br/politica/\d+")

In [4]:
crawler_politica.crawl()

getting page http://blogs.opovo.com.br/politica/
getting page http://blogs.opovo.com.br/politica/2019/04/10/jaques-wagner-diz-que-estilo-do-ciro-nao-ajuda-e-que-ex-candidato-tem-ciume/
getting page http://blogs.opovo.com.br/politica/2019/04/11/assembleia-do-rio-de-janeiro-aprova-porte-de-arma-para-deputados-estaduais/
getting page http://blogs.opovo.com.br/politica/2019/04/09/eduardo-bismarck-denuncia-deputado-por-estar-armado-na-ccj-da-camara/
getting page http://blogs.opovo.com.br/politica/2019/04/08/quem-e-o-novo-ministro-da-educacao-de-jair-bolsonaro/
getting page http://blogs.opovo.com.br/politica/2019/04/10/novo-ministro-debocha-de-cid-em-cerimonia-esse-ficou-menos-tempo-que-o-velez/
getting page http://blogs.opovo.com.br/politica/2019/04/11/deputado-do-psl-que-quer-proibir-games-violentos-defende-liberar-armas/
getting page http://blogs.opovo.com.br/politica/2019/04/10/jaques-wagner-diz-que-estilo-do-ciro-nao-ajuda-e-que-ex-candidato-tem-ciume/#respond
getting page http://blogs.

In [11]:
crawler_hardware = Crawler(os.path.join(os.pardir, "Data/corpora/hardware"), 50, "https://www.techspot.com/reviews/", "^https://www.techspot\.com/review/\d+")

In [10]:
crawler_hardware.crawl()

getting page https://www.techspot.com/reviews/
getting page https://www.techspot.com/review/1776-das-keyboard-4q/
getting page https://www.techspot.com/review/1777-geforce-1050ti-vs-radeon-570/
getting page https://www.techspot.com/review/1784-resident-evil-2-benchmarks/
getting page https://www.techspot.com/review/1781-geforce-rtx-2060-mega-benchmark/
getting page https://www.techspot.com/review/1807-asus-rog-strix-scar2-rtx-laptop/
getting page https://www.techspot.com/review/1811-nvidia-geforce-gtx-1060/
getting page https://www.techspot.com/review/1817-asus-zenbook-pro-14/
getting page https://www.techspot.com/review/1815-msi-gs75-stealth/
getting page https://www.techspot.com/review/1789-amd-radeon-vii/
getting page https://www.techspot.com/review/1813-geforce-gtx-1660-mega-benchmark/
getting page https://www.techspot.com/review/1799-geforce-gtx-1660-mega-benchmark/
getting page https://www.techspot.com/review/1791-amd-radeon-vii-mega-benchmark/
getting page https://www.techspot.c